# Datenvisualisierung

Datenvisualisierung ist der Prozess der Verwendung visueller Elemente wie Diagramme, Grafiken oder Karten zur Darstellung von Daten. Sie übersetzt komplexe, umfangreiche oder numerische Daten in eine visuelle Darstellung, die leichter zu verarbeiten ist.

## Voraussetzung für dieses Beispiel

Dieses Beispiel setzt eine MySQL Datenbank mit Namen DataVisualisation voraus. In Dateiverzeichnis in dem sich dieses Notebook befindet, ist ein SQL-Script vorhanden, mit dem diese Datenbank erstellt werden kann.

Weiter setzt dieses Notebook eine Python-Umgebung (Empfehlung Anaconda) voraus.

## Repetition Python und MySQL

In diesem Beispiel verwenden wir Daten aus einer MySQL-Datenbank die wir als Grafik darstellen wollen. Die hier verwendeten Tools (Matplotlib, Seaborn, Pandas) können jedoch genauso mit Daten aus csv-Files oder anderen Datenquellen umgehen.

### Daten aus einer Mysql Datenbank beschaffen

Um aus Python mit MySQL Datenbanken arbeiten zu können, muss das entsprechende Modul installiert sein.

> ./anaconda3/bin/pip install mysql-connector-python

Das kann auch aus dem Notebook heraus gemacht werden (siehe Zelle unten). Läuft das Notebook nicht in der von mir abgegbenen Lernumgebung, muss der Pfad zur Anaconda Installation angepasst werden.

In [ ]:
%%bash
/home/pkmlp/anaconda3/bin/pip install mysql-connector-python

In [ ]:
#
# Importieren der benötigten Python-Module
#

import mysql.connector


### Verbindung zur Datenbank aufbauen

In [ ]:
##
## Erstellen einer Verbindung zur Datenbank
##

user = "root" 
passwd = "pkmlp" 
host = "localhost" 
port = "3306"
db = "DataVisualisation"

dbConnection = mysql.connector.connect(host = host,
                                       port = port,
                                       user = user, 
                                       passwd = passwd, 
                                       db = db)


### Datenbank-Befehl aufbereiten und mittels Datenbank-Cursor absetzen

Das auszuführende SQL-Statement in einer Variablen aufbereiten.

In [ ]:
sqlStatement = "SELECT ProductName, ChangeDate, PriceAfter FROM PriceHistory ORDER BY ProductName, ChangeDate;"

Da bei der Abfrage einer relationalen Datenbanken immer eine Menge (leere Menge, eine Menge mit genau einem Attribut, oder eine Menge, die die gesamte Tabellen beinhaltet) zurückgibt, muss ein Cursor definiert werden, damit die Resultatmenge dann Zeile für Zeile abgearbeitet werden kann.

In [ ]:
dbCursor = dbConnection.cursor()

Das auszuführende SQL-Statement wird dem Cursor in der Variablen übergeben und vom Cursor ausgeführt. 

In [ ]:
dbCursor.execute(sqlStatement)

### Daten verarbeiten

Nun ist die Resultatmenge im Objekt dbCursor und wir können die wie in Python gewohnt bearbeiten.

In [ ]:
for DataRow in dbCursor:
    print("ProductName:", DataRow[0], "ChangeDate:", DataRow[1], "PriceAfter:", DataRow[2])

Hinweis: Ist ein Cursor in einer Schlaufe abgearbeitet, wird er aufgelöst.

### Verbindung zur Datenbank schliessen

Die Verbindung zur Datenbank nur so lange wie nötig aufrecht erhalten.

In [ ]:
dbConnection.close()

Damit ist die Repetition Python und MySQL abgeschlossen. Wir wissen, wie wir Daten aus einer MySQL-Datenbank in ein Python-Programm einlesen können.

## Daten visualisieren 

Python bietet mit den Modulen Pandas und Seaborn eine Vielzahl von Funktionen zur Verarbeitung von sogenannten DataFrames. Das sind im Wesentlichen Tabellen, was ja genau einem Result-Set eines SQL-Statements entspricht. Damit Daten aus beliebigen Datenquelllen verwendet werden können, verwenden wir in diesem Beispiel das Python Modul SQLAlchemy.


>SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL.

>It provides a full suite of well known enterprise-level persistence patterns, designed for efficient and high-performing database access, adapted into a simple and Pythonic domain language.

### Installieren der Python Module

Installieren der benötigten Python Module für Pandas, Seaborn und SQLAlchemy in unserem Python-Environment

In [ ]:
%%bash
/home/pkmlp/anaconda3/bin/pip install pandas
/home/pkmlp/anaconda3/bin/pip install sqlalchemy
/home/pkmlp/anaconda3/bin/pip install pymysql
/home/pkmlp/anaconda3/bin/pip install seaborn


### Importieren der Python Module

Importieren der benötigten Pythn Module für MySQL (resp. SQLAlchemy) und Pandas im Python-Programm.

In [ ]:
#
# Importieren der benötigten Python-Module
#

import sqlalchemy as sqla
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
%matplotlib inline


### Erstellen einer Verbindung zur Datenbank

Der Connection String sieht für SQLAlchemy etwas anders aus, da mit SQLAlchemy beliebige Datenbanken und andere Datenquellen (csv-Files) verwendet werden können. Die benötigten Paramater sind jedoch die gleichen.


In [ ]:
##
## Erstellen einer Verbindung zur Datenbank
##

user = "root" 
passwd = "pkmlp" 
host = "localhost" 
port = "3306"
db = "DataVisualisation"

dbEngine = sqla.create_engine('mysql+pymysql://' + user + ':' + passwd + '@' + host + ':' + port + '/' + db)
dbConnection = dbEngine.connect()


### SQL Statement aufbereiten und absetzen

Nachdem wir eine Verbindung zur Datenbank erstellt haben, können wir SQL-Statements absetzen. Diesmal wollen wir aber das Result-Set des SQL-Statements nicht Zeile für Zeile verarbeiten, sondern wir wollen es in einem sogenannten pandas DataFrame haben, das wir anschliessend sehr komfortabel mit pandas Funktionalität weiter verarbeiten (auswerten) können.

In [ ]:
#
# SQL Statement aufbereiten und absetzen
#
sqlStatement = 'SELECT ProductName, ChangeDate, PriceAfter FROM PriceHistory ORDER BY ProductName, ChangeDate;'
df = pd.DataFrame(pd.read_sql_query(sqlStatement, dbConnection))


### Datenbankverbindung schliessen

In [ ]:

#
# Da wir das ResultSet nun als DataFrame haben, schliessen wir die Verbindung zur Datenbank
# Das DataFrame bleibt uns dabei erhalten und wir können es beliebig weiterverarbeiten
#
dbConnection.close()

## Datenvisualisierung mit Pandas und Seaborn 

In [ ]:
#
# Zur Kontrolle schauen wir uns den Inhalt des DataFrames an
#
df

In [ ]:
#
# und die Datentypen des DataFrames
#
df.dtypes

Nun können wir dieses DataFrame (resp. die in diesem DataFrame enthaltenen Preisentwicklungen verschiedener Produkte) visualisieren

In [ ]:

plt.figure(figsize=(12, 6))
sns.lineplot(x='ChangeDate', y='PriceAfter', data=df, hue='ProductName', marker='X', markersize=9, linestyle=':')
plt.xlabel('ChangeDate')
plt.xticks(rotation=45, horizontalalignment='right')
plt.ylabel('PriceAfter')
plt.title('Preisänderungen über die Zeit')
plt.show()


Empfehlung: Schaut den Code mit der erstellten Grafik an. Damit sind die Parameter im Code ziemlich Slebsterklärend. Sonst helfen untenstehende Links, Google, Youtube oder ChatGPT gerne weiter 

# Wichtige Links

> https://www.anaconda.com/

> https://www.sqlalchemy.org/

> https://seaborn.pydata.org/

> https://pandas.pydata.org/


## That's All Folks